In [1]:
# Source: https://github.com/gillus/llm.c/blob/master/data_preparation.ipynb


In [15]:
# !pip install polars
# !pip install --upgrade 'typing_extensions>=4.8.0' pydantic-core ollama
# !pip install dotenv
# !pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 5.1 MB/s  0:00:01 eta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [matplotlib]6 [matplotlib]


In [1]:
import glob
import json
from tqdm import tqdm
import os
import json
import pandas as pd
import re
import polars as pl

In [2]:
# Exploring FineWeb Edu

import polars as pl
# big download, only run once
!wget https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/10BT/000_00000.parquet
x= pl.scan_parquet('./000_00000.parquet')
x.head().collect()

text,id,dump,url,file_path,language,language_score,token_count,score,int_score
str,str,str,str,str,str,f64,i64,f64,i64
"""The Independent Jane For all t…","""<urn:uuid:0d8a309d-25c5-405d-a…","""CC-MAIN-2013-20""","""http://austenauthors.net/the-i…","""s3://commoncrawl/crawl-data/CC…","""en""",0.97432,845,2.75,3
"""Taking Play Seriously By ROBIN…","""<urn:uuid:316c7af5-14e1-4d0b-9…","""CC-MAIN-2013-20""","""http://query.nytimes.com/gst/f…","""s3://commoncrawl/crawl-data/CC…","""en""",0.961459,1055,2.5625,3
"""How do you get HIV? HIV can be…","""<urn:uuid:a3e140cd-7f25-48c9-a…","""CC-MAIN-2013-20""","""http://www.childline.org.uk/Ex…","""s3://commoncrawl/crawl-data/CC…","""en""",0.966757,136,3.125,3
"""CTComms sends on average 2 mil…","""<urn:uuid:c337bcd8-6aa1-4f2d-8…","""CC-MAIN-2013-20""","""http://www.ctt.org/resource_ce…","""s3://commoncrawl/crawl-data/CC…","""en""",0.910602,3479,3.234375,3
"""Hold the salt: UCLA engineers …","""<urn:uuid:c0b175bb-65fb-420e-a…","""CC-MAIN-2013-20""","""http://www.environment.ucla.ed…","""s3://commoncrawl/crawl-data/CC…","""en""",0.924981,1115,2.8125,3


In [3]:
x= pl.scan_parquet(glob.glob('./*_00000.parquet'))
x = x.filter((pl.col('token_count')>100)&(pl.col('score')>3.)&(pl.col('language')=='en')&(pl.col('language_score')>0.95))
x = x.collect()

In [4]:
print(f'Number of texts: {x.shape[0]}, number of tokens {x["token_count"].sum()/1e6} M')


Number of texts: 125518, number of tokens 139.898675 M


In [6]:
# Only test [:5] rows for local CPU, 
# Recommend use google colab to run full steps after testing
x = x[:5]

In [7]:
# Let's build our own data annotator!

import pandas as pd
import ollama
import os
import time
import json
from dotenv import load_dotenv
from tqdm import tqdm  # Import tqdm for progress bar

load_dotenv()
# API_KEY = os.getenv("MY_API_KEY")

# if not API_KEY:
#     raise ValueError("Google API Key not found. Please set the GOOGLE_API_KEY environment variable or define it in the script.")

# genai.configure(api_key=API_KEY)
# MODEL_NAME = "gemini-1.5-flash-8b"

# Use ollama local model, make sure install and run "ollama [pull|run] qwen2.5:3b"
MODEL_NAME = "qwen2.5:3b"

ALLOWED_TOPICS = [
    "Arts & Humanities",
    "History & Archaeology",
    "Social Sciences",
    "Mathematics",
    "Physical Sciences",
    "Children entertrainment",    
    "Computer Science",
    "Engineering & Technology",
    "Life Sciences",
    "Health & Medicine",
    "Education Studies",
    "Business & Finance",
    "Law & Legal Studies",
    "Environmental Science & Sustainability",
    "Languages & Linguistics",
    "Daily Routines & Home Management",
    "Family & Interpersonal Relationships",
    "Hobbies, Leisure & Entertainment",
    "Personal Health, Wellness & Lifestyle", 
    "Work Life & Career", 
    "Consumer Experiences & Personal Finance",
    "Personal Journeys & Life Events",
    "Food & Culinary"    
]

EDUCATION_LEVEL = ["primary school", "middle school", "high school", "university degree", "PhD degree"]

def classify_text_with_gemini(text_to_classify):
    """
    Uses Ollama qwen2.5:3b to classify text into topic and complexity.

    Args:
        text_to_classify (str): The text content to classify.

    Returns:
        tuple: (topic, complexity) or (None, None) if an error occurs.
    """
    # Ensure text is not empty or just whitespace
    if not text_to_classify or text_to_classify.isspace():
        #print("Warning: Skipping empty or whitespace-only text.")
        return None, None

    max_length = 15000
    if len(text_to_classify) > max_length:
        text_to_classify = text_to_classify[:max_length] + "..." # Truncate

    prompt = f"""
Analyze the following text and determine its primary topic and the educational level typically required to understand it.

Text:
"{text_to_classify}"

Instructions:
1. Choose the *single best* topic from this list: {ALLOWED_TOPICS}
2. Choose the *single most appropriate* education level one would need to properly understand the text: {EDUCATION_LEVEL}
3. Provide your answer ONLY in the following JSON format:
   {{"topic": "SELECTED_TOPIC", "education": "SELECTED_EDUCATION"}}

Example Response:
{{"topic": "Science", "education": "high school"}}

Output ONLY the JSON object.
"""

    try:
        # Use local Ollama model
        response = ollama.generate(
            model=MODEL_NAME,
            prompt=prompt,
            stream=False,
            options={
                'temperature': 0.1  # Lower temperature for more deterministic classification
            }
        )
        
        raw_response_text = response['response'].strip()
        
        if raw_response_text.startswith("```json"):
            raw_response_text = raw_response_text[7:] # Remove ```json
        if raw_response_text.endswith("```"):
            raw_response_text = raw_response_text[:-3] # Remove ```
        raw_response_text = raw_response_text.strip() # Clean whitespace again

        # Parse the JSON response
        result = json.loads(raw_response_text)
        topic = result.get("topic")
        education = result.get("education")

        # Validate response against allowed categories
        if topic not in ALLOWED_TOPICS:
            print(f"Warning: Received invalid topic '{topic}'. Setting to None.")
            topic = None
        if education not in EDUCATION_LEVEL:
            print(f"Warning: Received invalid complexity '{education}'. Setting to None.")
            education = None

        return topic, education

    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON response: {raw_response_text}")
        return "Error: JSON Decode", "Error: JSON Decode"
    except ValueError as ve: # Handle potential errors from model generation (e.g., blocked content)
         print(f"Error: Ollama API returned ValueError (potentially blocked content or invalid response structure). Details: {ve}")
         return "Error: API Value", "Error: API Value"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "Error: API Call", "Error: API Call"

In [8]:
classify_text_with_gemini("The Copy Number Variation analysis of a DNA sequence can help characterize tumoroid cells")


('Health & Medicine', 'university degree')

In [10]:
classify_text_with_gemini("The itsy bitsy spider went up the water spout")


('Hobbies, Leisure & Entertainment', 'primary school')

In [10]:
len(x)

5

In [ ]:
import concurrent.futures
import pandas as pd
from tqdm import tqdm 

MAX_WORKERS = 8

label_categories = [None] * len(x)
label_degrees = [None] * len(x)



with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_index = {
        executor.submit(classify_text_with_gemini, text): i
        for i, text in enumerate(x['text'])
    }

    # for future in tqdm(concurrent.futures.as_completed(future_to_index), total=10000, desc="Classifying Text"):
    for future in tqdm(concurrent.futures.as_completed(future_to_index), total=5, desc="Classifying Text"):
        index = future_to_index[future]
        try:
            result_tuple = future.result()
            label_categories[index] = result_tuple[0]
            label_degrees[index] = result_tuple[1]
        except Exception as exc:
            print(f'Row {index} generated an exception: {exc}')
            label_categories[index] = "Error: Future Exception"
            label_degrees[index] = "Error: Future Exception"

In [12]:
x = x.with_columns([
    pl.Series("label_category", label_categories),
    pl.Series("label_degree", label_degrees)
])

In [13]:
x.write_parquet('simple_english.parquet')

In [ ]:
import matplotlib.pyplot as plt
plt.hist(x['label_category'])
plt.xticks(rotation=90)

In [ ]:
plt.hist(x['label_degree'])
plt.xticks(rotation=90)

In [17]:
# Generating synthetic data

In [ ]:
import json

def generate_synthetic_data(sample_text):
    """
    Uses Gemini to generate synthetic data from a sample text. The synthetic data is obtained by summarizing,
    paraphrasing, extracting keywords, and creating a creative variant of the text.

    Args:
        sample_text (str): The sample text on which to base the synthetic data generation.

    Returns:
        dict: A dictionary containing synthesized data with the following keys:
              - "summary": A succinct summary of the text.
              - "paraphrase": A reworded version of the text.
              - "keywords": A list of keywords extracted from the text.
              - "synthetic_variant": A creatively modified variant of the text.
              Returns error messages in the dictionary fields if an error occurs.
    """
    # Ensure text is not empty or just whitespace
    if not sample_text or sample_text.isspace():
        print("Warning: Skipping empty or whitespace-only text.")
        return {
            "summary": None,
            "paraphrase": None,
            "keywords": None,
            "synthetic_variant": None,
        }

    # Limit text length to avoid exceeding model limits (adjust as needed)
    max_length = 15000  # Example limit, adjust based on model token limits if needed
    if len(sample_text) > max_length:
        sample_text = sample_text[:max_length] + "..."  # Truncate

    prompt = f"""
You are an advanced language model designed to perform text augmentation. Given the following text, your task is to generate synthetic data by performing these four tasks:

1. **Summarization**: Provide a succinct summary of the text.
2. **Paraphrasing**: Rephrase the text while preserving its meaning.
3. **Synthetic Variant**: Generate one additional variant of the text with creative modifications (e.g. varying sentence structure or introducing subtle stylistic changes).

Please provide your answer ONLY in the following JSON format:
{{
    "summary": "Summarized version of the text.",
    "paraphrase": "Paraphrased version of the text.",
    "synthetic_variant": "Creative synthetic variant of the text."
}}

Example Response:
{{
    "summary": "A brief summary of the sample text.",
    "paraphrase": "A reworded version of the sample text.",
    "synthetic_variant": "An alternative version of the text with creative modifications."
}}

Text:
\"{sample_text}\"

Output ONLY the JSON object.
"""

    try:
        model = genai.GenerativeModel("gemini-1.5-flash")  # Replace with your actual model name
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.2,  # Adjust temperature to balance creativity and fidelity
                # You can also fine-tune the max output tokens if necessary, e.g.
                # max_output_tokens=150,
            )
        )

        raw_response_text = response.text.strip()
        # Remove markdown formatting if present
        if raw_response_text.startswith("```json"):
            raw_response_text = raw_response_text[7:]
        if raw_response_text.endswith("```"):
            raw_response_text = raw_response_text[:-3]
        raw_response_text = raw_response_text.strip()

        # Parse the JSON response
        result = json.loads(raw_response_text)
        # Optionally, validate that required keys are present
        for key in ["summary", "paraphrase", "synthetic_variant"]:
            if key not in result:
                print(f"Warning: Expected key '{key}' not found in the output. Setting its value to None.")
                result[key] = None

        return result

    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON response: {raw_response_text}")
        return {
            "summary": "Error: JSON Decode",
            "paraphrase": "Error: JSON Decode",
            "synthetic_variant": "Error: JSON Decode",
        }
    except ValueError as ve:
        print(f"Error: Gemini API returned ValueError. Raw response: {response.text if 'response' in locals() else 'N/A'}. Details: {ve}")
        return {
            "summary": "Error: API Value",
            "paraphrase": "Error: API Value",
            "synthetic_variant": "Error: API Value",
        }
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {
            "summary": "Error: API Call",
            "paraphrase": "Error: API Call",
            "synthetic_variant": "Error: API Call",
        }

In [ ]:
generate_synthetic_data(df['text'][0])

In [1]:
ALLOWED_TOPICS = [
    "Arts & Humanities",
    "History & Archaeology",
    "Social Sciences",
    "Mathematics",
    "Physical Sciences",
    "Children entertrainment",    
    "Computer Science",
    "Engineering & Technology",
    "Life Sciences",
    "Health & Medicine",
    "Education Studies",
    "Business & Finance",
    "Law & Legal Studies",
    "Environmental Science & Sustainability",
    "Languages & Linguistics",
    "Daily Routines & Home Management",
    "Family & Interpersonal Relationships",
    "Hobbies, Leisure & Entertainment",
    "Personal Health, Wellness & Lifestyle", 
    "Work Life & Career", 
    "Consumer Experiences & Personal Finance",
    "Personal Journeys & Life Events",
    "Food & Culinary",
    "Religion & Spirituality",
    "Children's Education",
    "Cultural Studies",
    'Entomology'
]
sorted(ALLOWED_TOPICS)

['Arts & Humanities',
 'Business & Finance',
 'Children entertrainment',
 "Children's Education",
 'Computer Science',
 'Consumer Experiences & Personal Finance',
 'Cultural Studies',
 'Daily Routines & Home Management',
 'Education Studies',
 'Engineering & Technology',
 'Entomology',
 'Environmental Science & Sustainability',
 'Family & Interpersonal Relationships',
 'Food & Culinary',
 'Health & Medicine',
 'History & Archaeology',
 'Hobbies, Leisure & Entertainment',
 'Languages & Linguistics',
 'Law & Legal Studies',
 'Life Sciences',
 'Mathematics',
 'Personal Health, Wellness & Lifestyle',
 'Personal Journeys & Life Events',
 'Physical Sciences',
 'Religion & Spirituality',
 'Social Sciences',
 'Work Life & Career']